# Initial Population

In [ ]:
import mpu.io
import numpy
import pandas as pd
import random
num_features=40
Population_size = 30
feature_size=5

# Defining the population size.
pop_size = (Population_size,num_features) 
def init():
    A=numpy.zeros((Population_size,num_features))
    for i in range(Population_size):
        j=0
        while(sum(A[i,:])<feature_size):
            r=numpy.random.randint(0,num_features-1) 
            A[i,r]=1 
    A=numpy.array(A)
    return(A)
new_population=init()

# Genetic Algorithm

In [ ]:
def cal_pop_fitness(new_population):
    
    for individual_index in range(new_population.shape[0]):
        individual=new_population[individual_index,:]         
        
        trial_feature_index = [index for index in range(len(individual)) if individual[index] == 1] 
        
        # Only those features with the curresponding gene of 1 are used for training and testing
        Xtrain_subset=numpy.asarray(Xtrain.iloc[:,trial_feature_index])
        
        #Objective Function ( Here it is a random forest regressor it can be any other mahcine learning function)
        RF=RandomForestRegressor()
        
        if len(X_train.columns)==0:
            fitt=0
        else:
            RF.fit(X_train, y_train)
            predictions = RF.predict(X_test)
            accuracy=r2_score(y_test, predictions)
            
        fitness.append(accuracy)
    return (fitness)

def elite_selection(pop, fitness, num_elite):
    idx   = numpy.argsort(fitness)
    fitness = numpy.array(fitness)[idx]
    pop = numpy.array(pop)[idx,:]
    elite_parents=numpy.empty((num_elite,pop.shape[1] ))
    fitness_elite_parents=[]
    num=0
    count=0
    while(count<num_elite):
        if (fitness[-num-1] not in fitness_elite_parents):
            elite_parents[count,:]=pop[-num-1,:]
            fitness_elite_parents.append(fitness[-num-1])
            count=count+1
        num=num+1
           
    return elite_parents

def select_mating_pool(pop, fitness, num_parents):
    
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = numpy.empty((num_parents, pop.shape[1]))
    sumfitness=sum(fitness)
    parents_fitness=[]
    num=0
    while(num<num_parents):
        rouletvalue=random.random()*sumfitness
        count=0
        while(rouletvalue>0):
            
            rouletvalue=rouletvalue-fitness[count]
            count=count+1
               
        max_fitness_idx = count-1
    
        parents_fitness.append(fitness[max_fitness_idx])
        parents[num, :] = pop[max_fitness_idx, :]
        num=num+1
                
    return parents

def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    idx=[]
    for k in range(0,offspring_size[0],2):
        # Index of the first parent to mate.
        
        parent1_idx=numpy.random.randint(0,parents.shape[0]-1)           
        # Index of the second parent to mate.
        parent2_idx=numpy.random.randint(0,parents.shape[0]-1)
        ## while (sum(parents[parent1_idx]==parents[parent2_idx])==40):  ## if we don't want the similar individual to mate
        while (parent1_idx==parent2_idx):
            parent2_idx=numpy.random.randint(0,parents.shape[0]-1)            
        idx.append([parent1_idx,parent2_idx])
        
        if (random.random()<Pc):

        # The point at which crossover takes place between two parents. Usually it is at the center.        
            cond=0
            while(cond==0):

                crossover_point1=numpy.random.randint(0,offspring_size[1]-1)
                crossover_point2=numpy.random.randint(0,offspring_size[1]-1)
                if (sum(parents[parent1_idx, crossover_point1:crossover_point2])==sum(parents[parent2_idx, crossover_point1:crossover_point2]) and (crossover_point1!= crossover_point2)):
                    cond=1

            offspring[k, 0:crossover_point1] = parents[parent1_idx, 0:crossover_point1]
            offspring[k, crossover_point1:crossover_point2] = parents[parent2_idx, crossover_point1:crossover_point2]
            offspring[k+1, 0:crossover_point1] = parents[parent2_idx, 0:crossover_point1]
            offspring[k+1, crossover_point1:crossover_point2] = parents[parent1_idx, crossover_point1:crossover_point2]
            offspring[k, crossover_point2:] = parents[parent1_idx, crossover_point2:]
            offspring[k+1, crossover_point2:] = parents[parent2_idx, crossover_point2:]
        else:
            offspring[k, :] = parents[parent1_idx, :]
            offspring[k+1, :] = parents[parent2_idx, :]
            
    return offspring

def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    if (random.random()<Pm):
        for idx in range(offspring_crossover.shape[0]):
            # The random value to be added to the gene.        
            ones=numpy.where(offspring_crossover[idx,:]==1)
            zeros=numpy.where(offspring_crossover[idx,:]==0)
            ones=ones[0]
            zeros=zeros[0]
            mut_position1=random.choice(ones)
            mut_position2=random.choice(zeros)
            offspring_crossover[idx,mut_position1]=0
            offspring_crossover[idx,mut_position2]=1
    return offspring_crossover


# Main Code

In [ ]:
globalfitnessmax=[] 
globalfitnessmean=[] 

for generation in range(num_generations):
     # Measuring the fitness of each chromosome in the population.
    fitness = cal_pop_fitness(new_population)
    
    globalfitnessmean.append(numpy.mean(fitness))
    globalfitnessmax.append(numpy.max(fitness))
    print("Generation:",generation)
    print("Best result : ", numpy.max(fitness))
    print("Mean Result:",numpy.mean(fitness))
    
    
    # Selecting the best parents in the population for mating.
    elites=elite_selection(new_population,fitness,num_elite)
    parents = select_mating_pool(new_population, fitness, 
                                       num_parents_mating)
    
    
     # Generating next generation using crossover.
    offspring_crossover = crossover(parents,
                                        offspring_size=(pop_size[0]-elites.shape[0], num_features))
    
     # Adding some variations to the offsrping using mutation.
    offspring_mutation = mutation(offspring_crossover)
    # Creating the new population based on the parents and offspring.
    new_population[0:elites.shape[0], :] = elites
    new_population[elites.shape[0]:, :] = offspring_mutation 


    

# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
fitness = cal_pop_fitness(new_population)
# Then return the index of that solution corresponding to the best fitness.
best_match_idx = numpy.where(fitness == numpy.max(fitness))

print("Best solution : ", new_population[best_match_idx, :])
print("Best solution fitness : ", fitness[best_match_idx])
